In [1]:
import os
import PyPDF2
from PyPDF2 import PdfReader
import pandas as pd
import re
import sys
from document_retrieval_BM25 import get_query_results
from doc_spell_checker import perform_correction
from tf_idf_retrieval import retrieve_top_files

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/devansharora/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/devansharora/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/devansharora/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/devansharora/anaconda3/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/devansharora/nltk_data

In [2]:
dataset_path = os.path.join(os.path.dirname(os.getcwd()), 'Dataset')
csv_path = os.path.join(os.path.dirname(os.getcwd()),'res','csv_etl_files')
pdf_paths = os.listdir(dataset_path)

In [3]:
csv_path

'/Users/devansharora/Desktop/IR/IR-Project/res/csv_etl_files'

In [5]:
print(pdf_paths)

['A2019-17.pdf', 'A2009-12.pdf', 'A2009-06.pdf', 'A2012-35.pdf', 'A2017-10_3.pdf', 'A2016-36_0.pdf', 'A2010-31.pdf', 'A2013-18_1.pdf', 'A2016-30_3.pdf', 'A2016-16_0.pdf', 'A2015-11.pdf', 'A-2009-25.pdf', 'A2017-12_5.pdf', 'A2019-10.pdf', 'A2016-32_0.pdf', 'A2017-14_1.pdf', 'Indian Institutes of Management Act 2017.pdf', 'A2010-23.pdf', 'A2017-16_2.pdf', 'A2017-13_1.pdf', 'A2010-16_0.pdf', 'A2014-40_0.pdf', 'A2018-02_0.pdf', 'A2013-14.pdf', 'A2011-05.pdf', 'A2011-11.pdf', 'A2018-24_0.pdf', 'A2014-37.pdf', 'A2016-11_1.pdf', 'A2018-04_0.pdf', 'A2009-07_1.pdf', 'A2010-42.pdf', 'A2016-49_1.pdf', 'A2011-15.pdf', 'A2018-25.pdf', 'A2017-15_2.pdf', 'A2014-30.pdf', 'A2014-18.pdf', 'A2016-17_1.pdf', 'A2017-26_1.pdf', 'A2013-20.pdf', 'A2016-4_1.pdf', 'A2014-1.pdf', 'A2013-23.pdf', 'A2014-17.pdf', 'A2017-02_2.pdf', 'A2018-17.pdf', 'A2018-03.pdf', 'A2013-26.pdf', 'A2017-20_0.pdf', 'A2010-1.pdf', 'A2014-7.pdf', 'A2013-25.pdf', 'A2014-10.pdf', 'A2013-30.pdf', 'A2014-6.pdf', 'A2017-22_1.pdf', 'A2016-2_

In [6]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        data = [(page.extract_text(), page_number + 1)
                for page_number, page in enumerate(reader.pages)]
    return data

def process_data_and_save_to_dataframe(pdf_path):
    data = extract_text_from_pdf(pdf_path)
    processed_data = []

    for page_text, page_no in data:
        lines = page_text.split('\n')
        title = ''
        for line in lines:
            line = line.strip()
            if line:
                if line.isupper() and line.strip('.').isalpha() and not line.isdigit():  
                    title = line
                else:
                    processed_data.append({'Title': title, 'Paragraph Text': line, 'Page No': page_no})

    dataframe = pd.DataFrame(processed_data)
    return dataframe

In [7]:
def split_text(text, max_length=500, overlap=50, min_final_chunk_length=150):
    # If the text is shorter than the max length, return it as is
    if len(text) <= max_length:
        return [text]
    
    chunks = []
    start_index = 0
    
    while start_index < len(text):
        # End index is either the max length of the chunk or the end of the text
        end_index = min(start_index + max_length, len(text))
        
        # If we are at the end of the text, append the chunk and break
        if end_index == len(text):
            chunks.append(text[start_index:end_index])
            break
        
        # Find the last space in the text to potentially split at
        split_index = text.rfind(' ', start_index, end_index + overlap)
        
        # If no space found, or if the split index is before the start index,
        # just split at the max length without an overlap
        if split_index <= start_index:
            split_index = end_index
        
        # Append the chunk up to the split index
        chunks.append(text[start_index:split_index])
        
        # Update the start index to continue from the split index
        start_index = split_index

    # After splitting, if the last chunk is smaller than the minimum size, merge it with the previous chunk
    if len(chunks) > 1 and len(chunks[-1]) < min_final_chunk_length:
        chunks[-2] += chunks[-1]  # Merge the last chunk with the second-to-last chunk
        chunks = chunks[:-1]  # Remove the last chunk since it's now merged

    return chunks


In [8]:
def combining_text_info(dataframe):
    dataframe_dict = dataframe.to_dict(orient='list')
    dataframe_dict_ret = {}
    paragraph_start_pattern = re.compile(r'^(\d+\.\s+.+)')
    for i in dataframe_dict.keys():
        dataframe_dict_ret[i] = []
    current_combined_text = ""
    current_paragraph_index = None
    for index, row in dataframe.iterrows():
        text = str(row['Paragraph Text']).strip()
        if paragraph_start_pattern.match(text):
            if current_paragraph_index is  None:
                current_paragraph_index = index
            else:
                split_combined_text = split_text(current_combined_text)
                title_lst = [dataframe_dict['Title'][current_paragraph_index]]*len(split_combined_text)
                page_info_lst = [dataframe_dict['Page No'][current_paragraph_start_index]]*len(split_combined_text)
                dataframe_dict_ret['Title'].extend(title_lst)
                dataframe_dict_ret['Paragraph Text'].extend(split_combined_text)
                dataframe_dict_ret['Page No'].extend(page_info_lst)
            current_combined_text = ""
            current_paragraph_start_index = index
            
        current_combined_text += text
    
    if len(current_combined_text) !=0:
        dataframe_dict_ret['Title'].append(dataframe_dict['Title'][current_paragraph_index])
        dataframe_dict_ret['Paragraph Text'].append(current_combined_text)
        dataframe_dict_ret['Page No'].append(dataframe_dict['Page No'][index])
    combined_dataframe = pd.DataFrame.from_dict(dataframe_dict_ret)
    return combined_dataframe

In [11]:
from tqdm import tqdm
def pipeline(dataset_path,pdf_paths,csv_path):
    for i in tqdm(range(len(pdf_paths))):
        main_path = dataset_path+'/'+pdf_paths[i]
        df = process_data_and_save_to_dataframe(main_path)
        com_df = combining_text_info(df)
        file_path = csv_path + '/'+pdf_paths[i][:-3]+'csv'
        com_df.to_csv(file_path)


In [12]:
pipeline(dataset_path,pdf_paths,csv_path)

100%|███████████████████████████████████████████| 77/77 [00:36<00:00,  2.12it/s]


In [15]:
from doc_spell_checker import perform_correction
from tqdm import tqdm
def spell_correction_pipeline(csv_path):
    csv_files = os.listdir(csv_path)
    for i in tqdm(range(len(csv_files))):
        csv_full_path = csv_path+'/'+csv_files[i]
        if csv_full_path[-3:] != "csv":
            continue
        df = pd.read_csv(csv_full_path)
        rows = len(df)
        dict_df = df.to_dict(orient='list')
        for j in range(rows):
            dict_df['Paragraph Text'][j] = perform_correction(dict_df['Paragraph Text'][j])
        df = pd.DataFrame.from_dict(dict_df)
        df.to_csv(csv_full_path,index=True)


In [16]:
spell_correction_pipeline(csv_path)

100%|███████████████████████████████████████████| 78/78 [00:46<00:00,  1.67it/s]


In [9]:
import csv
from collections import defaultdict
import os
from tqdm import tqdm
def extract_filename(file_path):
    # Use the os.path.basename function to get the filename from the file path
    filename = os.path.basename(file_path)
    return filename

def tokenize(text):
    # Basic tokenizer, could be enhanced with NLTK or similar libraries
    return text.lower().split()

def create_inverted_index(csv_files, inverted_index_path):
    inverted_index = defaultdict(set)

    for i in tqdm(range(len(csv_files))):
        if csv_files[i][-3:] != 'csv':
            continue
        df = pd.read_csv(csv_files[i])
        for j in range(len(df)):
            paragraph = str(df['Paragraph Text'].iloc[j])
            if paragraph:
                for word in tokenize(paragraph):
                    # Use the filename and row index as a unique identifier for the paragraph
                    doc_id = extract_filename(csv_files[i])
                    inverted_index[word].add(csv_files[i])
    
    with open(inverted_index_path, 'wb') as file:
        pickle.dump(inverted_index, file)
    return inverted_index



# Example usage:
# Assuming 'csv_files' is a list of paths to CSV files and 'inverted_index_path' is the path where the index should be saved.
# inverted_index = create_inverted_index(csv_files, inverted_index_path)
# Note: Actual file reading/writing would not work in this environment as file access is restricted.
# This code is provided for illustrative purposes and should be run in a local Python environment.


In [10]:
csv_files = os.listdir(csv_path)
csv_files = [csv_path+"/"+csv for csv in csv_files]
print(csv_files[0])
inverted_index_path = os.path.join(os.path.dirname(os.getcwd()),'res')
inverted_index_path = inverted_index_path+'/inverted_index.pkl'
inverted_index = create_inverted_index(csv_files,inverted_index_path)

/Users/devansharora/Desktop/IR/IR-Project/res/csv_etl_files/A2015-22.csv


100%|██████████████████████████████████████████| 78/78 [00:00<00:00, 181.02it/s]


8111

In [47]:
inverted_index_path = os.path.join(os.path.dirname(os.getcwd()),'res')
inverted_index_path = inverted_index_path+'/inverted_index.pkl'
inverted_index = create_inverted_index(inverted_index_path)

100%|████████████████████████████████| 15783/15783 [00:00<00:00, 6012597.64it/s]


dict_keys(['s', 'h', 'o', 'r', 't', ' ', 'i', 'l', 'e', 'c', 'm', 'n', 'd', 'f', 'a', 'b', 'p', 'v', 'j', 'u', 'y', 'w', 'g', 'x', 'z', 'k', 'q'])

In [3]:
csv_files = os.listdir(csv_path)
file_paths = []
for i in range(len(csv_files)):
    if csv_files[i][-3:] == 'csv':
        file_paths.append(csv_path+'/'+csv_files[i])

In [12]:
import pickle
query="What is service charge?"
csv_path = os.path.join(os.path.dirname(os.getcwd()),'res','csv_etl_files')
csv_files = os.listdir(csv_path)
csv_files = [csv_path+'/'+csv for csv in csv_files]
inverted_index_path = os.path.join(os.path.dirname(os.getcwd()),'res','inverted_index.pkl')
with open(inverted_index_path, 'rb') as file:
    inverted_index = pickle.load(file)
top_files = retrieve_top_files(query,inverted_index,10)
print(top_files)
results = get_query_results(query,top_files,5)

['/Users/devansharora/Desktop/IR/IR-Project/res/csv_etl_files/A2017-13_1.csv', '/Users/devansharora/Desktop/IR/IR-Project/res/csv_etl_files/A2011-15.csv', '/Users/devansharora/Desktop/IR/IR-Project/res/csv_etl_files/A2017-27_1.csv', '/Users/devansharora/Desktop/IR/IR-Project/res/csv_etl_files/A2017-15_2.csv', '/Users/devansharora/Desktop/IR/IR-Project/res/csv_etl_files/A2017-26_1.csv', '/Users/devansharora/Desktop/IR/IR-Project/res/csv_etl_files/A2016-11_1.csv', '/Users/devansharora/Desktop/IR/IR-Project/res/csv_etl_files/A2017-12_5.csv', '/Users/devansharora/Desktop/IR/IR-Project/res/csv_etl_files/A2017-14_1.csv', '/Users/devansharora/Desktop/IR/IR-Project/res/csv_etl_files/A2017-10_3.csv', '/Users/devansharora/Desktop/IR/IR-Project/res/csv_etl_files/A2014-17.csv']


In [13]:
print(results)

{'query': 'service charge', 'retrieval_results': ['tax council constitute under the provision of article of the fund mean the good and service tax compensation fund refer to in section input tax in relation to a taxable person mean si ce charge on any supply of good or service or both make to ce charge on import of good and include the ce payable on reverse charge ba sish integrate good and service tax act mean the integrate good and service tax act of integrate tax mean the integrate good', 'good and service st ax act and the good and service tax compensation to state act if char ged separately by any amount that the supplier be liable to pay in relation to such supply but which have been incur red by the recipient of the supply and not include in the price actually pa id or payable for the goods or service or both c incidental expense include commission and pack charge by the supplier to the recipient of a supply and any amount charge for anything do by the supplier in respect of the